In [12]:
import pandas as pd
from scipy import sparse
from preprocessing.duration_matrix import DurationSparseMatrix
from scipy.spatial.distance import cosine

In [3]:
CONST_PATH = '../data/postprocessing/'

In [14]:
users_genres = pd.read_csv(f'{CONST_PATH}users_genres.csv', index_col='user_uid')
users_genres.head()

,игровой,шоу,музыка,комедия,мелодрама,драма,комедийный,фэнтезийный,боевик,приключение,...,молодёжь,детский,ситком,индонезия,психологический,увлечение,досуг,концерт,скетчкома,интервью
user_uid,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,400,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,27683,12981,12324,5154,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,0,0,0,900,6920,10666,1559,0,14166,0,...,0,0,0,0,0,0,0,0,0,0
43,0,0,0,599,61,61,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45,0,0,0,1470,1938,4257,293,0,6029,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
users_types = pd.read_csv(f'{CONST_PATH}user_content_types.csv', index_col=0)
users_types.head()

,serial_with_season,movie,serial_without_season
1,0,6720,0
4,30169,23638,0
25,665,23969,0
43,0,660,0
45,0,14370,0


In [16]:
USER_ID = 4
genres_similar_users = users_genres.drop(USER_ID, axis=0).apply(
    lambda row: cosine(users_genres.loc[USER_ID], row), axis=1)

In [17]:
genres_similar_users = genres_similar_users.sort_values().head(500)

In [20]:
most_similar_index = genres_similar_users.index
most_similar_users_types = users_types.loc[most_similar_index]

,serial_with_season,movie,serial_without_season
user_uid,,,
12109,23797,24036,0
1030299,25555,11731,0
1061216,1845,590,0
174178,26173,66217,0
594868,33617,584,0
...,...,...,...
9085,0,12253,0
421989,0,36819,0
151976,0,8350,0


In [24]:
most_similar_users_types.reset_index().iloc[:, 1:]

,serial_with_season,movie,serial_without_season
0,23797,24036,0
1,25555,11731,0
2,1845,590,0
3,26173,66217,0
4,33617,584,0
...,...,...,...
495,0,12253,0
496,0,36819,0
497,0,8350,0
498,0,15571,0


In [25]:
type_similar_users = most_similar_users_types.apply(
    lambda row: cosine(most_similar_users_types.reset_index().iloc[USER_ID, 1:], row), axis=1)

In [27]:
type_similar_users = type_similar_users.sort_values(kind='mergesort').head(100)

In [42]:
history = pd.read_csv(f'{CONST_PATH}watch_history.csv').drop(columns='Unnamed: 0')
history.head()

,user_uid,content_uid,second
0,1,11882.0,627
1,1,26174.0,5693
2,1,26719.0,400
3,4,2829.0,12
4,4,3836.0,64


In [43]:
content = pd.read_csv(f'{CONST_PATH}content.csv', index_col='content_uid')
content.head()

,name,type,serial_id,genres,duration_seconds
content_uid,,,,,
9882,"""Шоу ""Студия ""Союз""",serial_with_season,NaN,"Игровое шоу,Музыка",88765.0
19870,#СидЯдома,serial_with_season,NaN,Комедия,13253.0
2312,(Не)жданный принц,movie,NaN,"Мелодрама,Комедия",4983.0
12655,1+1,movie,NaN,"Драма,Комедия",6452.0
25771,1+1. Нарушая правила,movie,NaN,Комедийная драма,6323.0


In [44]:
history = history.merge(content.loc[:,'duration_seconds'], how='inner', on='content_uid')

user_uid
594868     0.000000e+00
161977     1.690356e-08
1225305    1.551650e-07
322459     6.512383e-07
196358     1.117449e-05
               ...     
911322     1.935918e-03
793110     2.833149e-03
608006     3.140983e-03
1107083    3.149060e-03
407484     5.198933e-03
Length: 100, dtype: float64